<a href="https://colab.research.google.com/github/karaage0703/karaage-ai-book/blob/master/ch02/02_karaage_ai_book_image_classification_on_camera.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# カメラでの画像分類


学習させたモデルを使ってカメラ画像に対して画像分類をします。

注：このノートブックはハードウェアアクセラレータをGPUよりNone(CPU)で実行した方がレスポンスが良い場合もあります。

## 事前準備

In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
print(tf.__version__)

In [ ]:
from tensorflow.keras.preprocessing.image import img_to_array, load_img
import numpy as np
import time

モデルとラベルをGoogle Driveからコピーします。

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!cp '/content/drive/My Drive/my_model.h5' './my_model.h5'
!cp '/content/drive/My Drive/labels.txt' './labels.txt'

In [ ]:
!ls -lh

## AIモデルとラベルの読み込み

MNISTベースのAIモデルを読み込んで中身を確認します

In [ ]:
keras_model = tf.keras.models.load_model("my_model.h5")
keras_model.summary()

ラベルを読み込みます

In [ ]:
labels = []
with open('./labels.txt','r') as f:
  for line in f:
    labels.append(line.rstrip())
print(labels)

NUM_CLASSES = len(labels)

## 画像分類デモの実行

Webカメラを使うJavascriptのコードをダウンロードします。

In [ ]:
!wget https://raw.githubusercontent.com/karaage0703/karaage-ai-book/master/util/colab_camera.py

コードを読み込みます。

In [ ]:
import colab_camera

処理を行うコールバック関数を定義します。

In [ ]:
import IPython
from google.colab import output
import cv2
import numpy as np
from PIL import Image
from io import BytesIO
import base64

def run(img_str):
  #decode to image
  decimg = base64.b64decode(img_str.split(',')[1], validate=True)
  decimg = Image.open(BytesIO(decimg))
  decimg = np.array(decimg, dtype=np.uint8); 
  decimg = cv2.cvtColor(decimg, cv2.COLOR_BGR2RGB)

  # image classify
  img = cv2.resize(decimg, (64, 64))
  img = img_to_array(img)
  img = img / 255.0
  img = img[np.newaxis]
  img = np.asarray(img)
  preds = keras_model.predict(img)
  result = np.argmax(preds[0])

  # draw output
  out_img = decimg
  cv2.putText(out_img, labels[result], (10,50), \
      cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 1, cv2.LINE_AA)

  #encode to string
  _, encimg = cv2.imencode(".jpg", out_img,
      [int(cv2.IMWRITE_JPEG_QUALITY), 80])
  img_str = encimg.tobytes()
  img_str = ("data:image/jpeg;base64," +
      base64.b64encode(img_str).decode('utf-8'))
  return IPython.display.JSON({'img_str': img_str})

output.register_callback('notebook.run', run)

デモを実施します。

In [ ]:
colab_camera.use_cam()

# 参考リンク

- https://qiita.com/a2kiti/items/f32de4f51a31d609e5a5